In [17]:
import dask
import numpy as np
import plotly.express as px
import xarray as xr
from dask.distributed import Client
from plotly.subplots import make_subplots
from SALib import ProblemSpec
from seapopym.configuration.no_transport.parameter import ForcingParameters, ForcingUnit, KernelParameters

from seapopym_optimization import wrapper

User can modify this :


In [ ]:
STATION = "Hot"
SAMPLE_NUMBER = 1

In [19]:
client = Client()
client

/Users/adm-lehodey/Documents/Workspace/Projects/Seapopym-optimisation/.venv/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 55545 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:55545/status,
Dashboard: http://127.0.0.1:55545/status,Workers: 4
Total threads: 12,Total memory: 48.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:55546,Workers: 4
Dashboard: http://127.0.0.1:55545/status,Total threads: 12
Started: Just now,Total memory: 48.00 GiB
Comm: tcp://127.0.0.1:55558,Total threads: 3
Dashboard: http://127.0.0.1:55560/status,Memory: 12.00 GiB
Nanny: tcp://127.0.0.1:55549,


Load forcing.


In [20]:
time_start, time_end = "2005-01-01", "2015-01-01"

data = xr.open_dataset(f"../1_data_processing/1_1_Forcing/products/{STATION}_cmems.zarr", engine="zarr")
data["time"].attrs = {"axis": "T"}
data = data.sel(time=slice(time_start, time_end))
_ = data.load()

cafe_npp = xr.open_dataset(f"../1_data_processing/1_1_Forcing/products/{STATION}_cafe.zarr", engine="zarr")
cafe_npp = cafe_npp.sel(time=slice(time_start, time_end))
cafe_npp = cafe_npp.dropna("time")
cafe_npp = cafe_npp.resample(time="D").interpolate("linear")
cafe_npp.time.attrs["axis"] = "T"
_ = cafe_npp.load()

In [21]:
data

<xarray.Dataset> Size: 643kB
Dimensions:              (latitude: 1, longitude: 1, time: 3653, depth: 3)
Coordinates:
  * depth                (depth) int64 24B 0 1 2
  * latitude             (latitude) float64 8B 22.75
  * longitude            (longitude) int64 8B -158
  * time                 (time) datetime64[ns] 29kB 2005-01-01 ... 2015-01-01
Data variables: (12/13)
    T                    (latitude, longitude, time, depth) float64 88kB 24.7...
    U                    (latitude, longitude, time, depth) float64 88kB -0.0...
    V                    (latitude, longitude, time, depth) float64 88kB 0.15...
    mnkc_epi             (latitude, longitude, time) float64 29kB 0.2904 ... ...
    mnkc_hmlmeso         (latitude, longitude, time) float64 29kB 0.8233 ... ...
    mnkc_lmeso           (latitude, longitude, time) float64 29kB 2.403 ... 1...
    ...                   ...
    mnkc_mumeso          (latitude, longitude, time) float64 29kB 0.5779 ... ...
    mnkc_umeso           (latitude, longitude, time) float64 29kB 0.79 ... 0....
    npp                  (latitude, longitude, time) float64 29kB 221.1 ... 3...
    pelagic_layer_depth  (latitude, longitude, time, depth) float64 88kB 107....
    zeu                  (latitude, longitude, time) float64 29kB 71.63 ... 7...
    zooc                 (latitude, longitude, time) float64 29kB 0.2527 ... ...
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-10-17
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Global ocean low and mid trophic levels biomass content hin...

In [22]:
cafe_npp

<xarray.Dataset> Size: 58kB
Dimensions:    (time: 3653, latitude: 1, longitude: 1)
Coordinates:
  * latitude   (latitude) float64 8B 22.75
  * longitude  (longitude) int64 8B -158
  * time       (time) datetime64[ns] 29kB 2005-01-01 2005-01-02 ... 2015-01-01
Data variables:
    CAFE       (time, latitude, longitude) float64 29kB 349.7 353.3 ... 332.5
Attributes:
    Conventions:  CF-1.8
    History:      2023-01-02 08:55:09.514113 Python
    Keywords:     Primary Production
    PI Contact:   tryankeogh@csir.co.za
    PI Name:      Thomas Ryan-Keogh

In [23]:
forcing_parameter_initial = ForcingParameters(
    temperature=ForcingUnit.from_dataset(forcing=data, name="T", resolution=0.08333, timestep=1),
    primary_production=ForcingUnit.from_dataset(cafe_npp, name="CAFE", resolution=0.08333, timestep=1),
)

2025-03-13 11:47:10,664 :: Seapodym ::  WARNING ::
|	CAFE unit is milligram / day / meter ** 2, it will be converted to kilogram / day / meter ** 2.



Generate initial conditions.


In [24]:
fg_parameters = wrapper.FunctionalGroupGeneratorNoTransport(
    [[0, 0, 0.1668, 10.38, -0.11, 150, -0.15]],
)
model = wrapper.model_generator_no_transport(
    fg_parameters=fg_parameters,
    forcing_parameters=forcing_parameter_initial,
    kernel_parameters=KernelParameters(compute_initial_conditions=True),
)
model.run()
initial_conditions = model.export_initial_conditions()

Model generation used in sensibility analysis.


In [25]:
FORCING_PARAMETERS = ForcingParameters(
    temperature=ForcingUnit.from_dataset(data, name="T", resolution=0.08333, timestep=1),
    primary_production=ForcingUnit.from_dataset(cafe_npp, name="CAFE", resolution=0.08333, timestep=1),
    initial_condition_biomass=ForcingUnit.from_dataset(
        initial_conditions, name="initial_condition_biomass", resolution=0.08333, timestep=1
    ),
    initial_condition_production=ForcingUnit.from_dataset(
        initial_conditions, name="initial_condition_production", resolution=0.08333, timestep=1
    ),
)


def wrapper_model_generator_no_transport(fg_parameters):
    fg_parameters = wrapper.FunctionalGroupGeneratorNoTransport(np.array([fg_parameters]))
    return wrapper.model_generator_no_transport(
        fg_parameters=fg_parameters,
        forcing_parameters=FORCING_PARAMETERS,
    )

2025-03-13 11:47:10,925 :: Seapodym ::  WARNING ::
|	CAFE unit is milligram / day / meter ** 2, it will be converted to kilogram / day / meter ** 2.



In [26]:
model = wrapper_model_generator_no_transport([0, 0, 0.1668, 10.38, -0.11, 150, -0.15])
model.run()

In [27]:
float(model.export_biomass().isel(functional_group=0, latitude=0, longitude=0).var().to_numpy())

2.6084412623590297e-09

In [28]:
np.var(model.export_biomass().isel(functional_group=0, latitude=0, longitude=0).to_numpy())

2.6084412623590297e-09

Setup sensitivity analysis structure.


In [29]:
sp = ProblemSpec(
    {
        "names": [
            "energy_transfert",
            "tr_max",
            "tr_rate",
            "inv_lambda_max",
            "inv_lambda_rate",
        ],
        "groups": None,
        "bounds": [
            [0, 1],
            [0.1, 50],
            [-0.5, -0.0001],
            [0.1, 500],
            [-0.5, -0.0001],
        ],
        "outputs": ["mean", "variance"],
    }
)

Create the cost function.


In [30]:
@dask.delayed
def cost_function(x: np.ndarray):
    energy_transfert, tr_max, tr_rate, inv_lambda_max, inv_lambda_rate = x.T
    fg_parameters = [
        0,
        0,
        energy_transfert,
        tr_max,
        tr_rate,
        inv_lambda_max,
        inv_lambda_rate,
    ]
    model = wrapper_model_generator_no_transport(fg_parameters)

    model.run()
    biomass_forcing = model.export_biomass().sel(time=slice(time_start, time_end))

    mean = float(biomass_forcing.mean())
    variance = float(biomass_forcing.var())

    return mean, variance

# SOBOL SENSITIVITY ANALYSIS


In [31]:
param_values = sp.sample_sobol(SAMPLE_NUMBER)

/Users/adm-lehodey/Documents/Workspace/Projects/Seapopym-optimisation/.venv/lib/python3.12/site-packages/scipy/stats/_qmc.py:993: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)


In [35]:
param_values.samples

array([[ 3.21904618e-01,  4.37065276e+01, -4.70300926e-01,
         1.28106061e+02, -4.55784328e-01],
       [ 6.05299165e-01,  4.37065276e+01, -4.70300926e-01,
         1.28106061e+02, -4.55784328e-01],
       [ 3.21904618e-01,  2.91671999e+01, -4.70300926e-01,
         1.28106061e+02, -4.55784328e-01],
       ...,
       [ 6.14324178e-01,  2.90235773e+01, -2.19270181e-01,
         3.93105578e+02, -5.45894348e-02],
       [ 6.14324178e-01,  2.90235773e+01, -2.19270181e-01,
         4.75723798e+02, -2.41710436e-02],
       [ 6.14324178e-01,  2.90235773e+01, -2.19270181e-01,
         4.75723798e+02, -5.45894348e-02]])

In [ ]:
res = [cost_function(param) for param in param_values.samples]
res = client.compute(res)
res = client.gather(res)
sp.set_results(np.asarray(res))

In [ ]:
sp.analyze_sobol()

mean_si, variance_si = sp.to_df()

In [ ]:
def plot_sobol(mean, variance, title):
    def generate_fig(total_si, first_si, second_si):
        fig_total = px.bar(total_si, x=total_si.index, y="ST", error_y="ST_conf", orientation="v")
        fig_first = px.bar(first_si, x=first_si.index, y="S1", error_y="S1_conf", orientation="v")
        fig_second = px.bar(second_si, x=second_si.index.map(str), y="S2", error_y="S2_conf", orientation="v")
        return fig_total, fig_first, fig_second

    fig_total_mean, fig_first_mean, fig_second_mean = generate_fig(*mean)
    fig_total_var, fig_first_var, fig_second_var = generate_fig(*variance)

    figure_mean = (
        make_subplots(
            rows=2,
            cols=3,
            # shared_yaxes=True,
            shared_xaxes=True,
            subplot_titles=(
                "Total Sobol indice : MEAN",
                "1st Sobol indice : MEAN",
                "2nd Sobol indice : MEAN",
                "Total Sobol indice : VARIANCE",
                "1st Sobol indice : VARIANCE",
                "2nd Sobol indice : VARIANCE",
            ),
            specs=[
                [{"type": "bar"}, {"type": "bar"}, {"type": "bar"}],
                [{"type": "bar"}, {"type": "bar"}, {"type": "bar"}],
            ],
            vertical_spacing=0.05,
            horizontal_spacing=0.05,
        )
        .add_trace(fig_total_mean.data[0], row=1, col=1)
        .add_trace(fig_first_mean.data[0], row=1, col=2)
        .add_trace(fig_second_mean.data[0], row=1, col=3)
        .add_trace(fig_total_var.data[0], row=2, col=1)
        .add_trace(fig_first_var.data[0], row=2, col=2)
        .add_trace(fig_second_var.data[0], row=2, col=3)
    )
    figure_mean.update_layout(height=800, width=1200, title_text=title)
    return figure_mean

In [ ]:
final_figure = plot_sobol(mean_si, variance_si, f"Sobol indices for mean biomass and variance in {STATION}")

In [ ]:
final_figure

In [ ]:
final_figure.write_html(f"./{STATION}_sobol_indice.html")